In [ ]:
import json
import requests
import datetime as dt
import pandas as pd
import urllib.parse
import plotly.express as px

from tqdm import tqdm

import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import os

In [ ]:
start_date = dt.date.today() + dt.timedelta(days=-90)
end_date = dt.date.today() + dt.timedelta(days=-1)

with open(os.getenv('USER_PATH'), 'r') as f:
    config = json.load(f)

config['database'] = config['database'] 

engine = sqlalchemy.create_engine('postgresql://'+config['user']+':'+config['password']+'@'+config['host']+':'+str(config['port'])+'/'+config['database'], pool_pre_ping=True)a

In [ ]:
df_sf = pd.read_csv('./sf_export.csv')
df_sf = df_sf[[y for y in df_sf.columns[0:10]]]

In [ ]:
# create blocks
days_per_block = 20
dates = [y.date() for y in pd.date_range(start_date, end_date)]
# orders
orders = []
for x in tqdm(range(0, len(dates), days_per_block)):
	this_start_date = dates[x]
	try:
		this_end_date = dates[x + days_per_block - 1]
	except:
		this_end_date = max(dates)
	q = f'''SELECT
			cu.id as company_user_id,
			cu.first_name,
			cu.last_name,
			cu.email,
			dc.name AS company_name,
			isc.industry_category_summarized_name,
			br.business_region_name,
			co.created_dt as order_created_dt,
			co.id as company_order_id,
			fos.company_order_shift_id
		FROM
			dds.dl_live_core_company_user cu
		JOIN
			cds.fact_company_order co
			ON cu.id = co.created_by_id
		JOIN
			cds.dim_company dc
			ON cu.company_id = dc.company_id
		JOIN
			cds.dim_industry_subcategory isc
			ON dc.industry_subcategory_id = isc.industry_subcategory_id
		JOIN
			cds.fact_company_order_shift fos
			ON co.id = fos.company_order_id
		JOIN
			cds.dim_job_category jc
			ON fos.job_category_id = jc.job_category_id
		JOIN
			cds.dim_job_location jl
			ON fos.job_location_id = jl.job_location_id
		JOIN
			cds.dim_business_region br
			ON jl.business_region_id = br.business_region_id
		WHERE
			co.created_dt BETWEEN '{this_start_date}' AND '{this_end_date}';'''
	tmp = pd.read_sql(q, engine)
	orders.append(tmp)
orders = pd.concat(orders).reset_index(drop=True)

100%|██████████| 5/5 [00:07<00:00,  1.54s/it]


In [ ]:
metrics = ['company_order_id', 'company_order_shift_id']
dims = [y for y in orders if y not in metrics]
orders.groupby(dims)[metrics].nunique().reset_index(drop=False).rename(columns={y:f'unique_{y[:-3]}s' for y in metrics}).to_excel('./user_orders_detail.xlsx')

In [ ]:
len(orders.groupby(dims)[metrics].nunique().reset_index(drop=False).rename(columns={y:f'unique_{y[:-3]}s' for y in metrics}))

24592